In [38]:
from langchain_community.document_loaders import UnstructuredPowerPointLoader
def pptx_parser(file_path):
    """
    Parses a PowerPoint file and returns its content as a list of documents.

    Args:
        file_path (str): The path to the PowerPoint file.
        
    Returns:
        list: A list of documents extracted from the PowerPoint file.
    """
    loader = UnstructuredPowerPointLoader(file_path)
    documents = loader.load()
    return documents
    

In [39]:
import os
available_extentions = ['pptx', 'ppt']

def parse_file(file_path, extentions=available_extentions):
    """
    Parses a file based on its extension and returns its content.
    """
    # Check if file exists
    if not os.path.exists(file_path):
        raise FileNotFoundError(f"File not found: {file_path}")
    # get the file's extension
    file_extension = file_path.split('.')[-1].lower()
    if file_extension not in extentions:
        raise ValueError(f"Unsupported file extension: {file_extension}")
    if file_extension in ['pptx', 'ppt']:
        return pptx_parser(file_path)
    else:
        raise ValueError(f"Unsupported file extension: {file_extension}")

In [ ]:
file_path = "../data/documents/Introduction to generative AI.pptx"
documents = parse_file(file_path)
for doc in documents:
    print(doc)

FileNotFoundError: File not found: ../data/Introduction to generative AI.pptx

In [1]:
# chunking the documents
from langchain.text_splitter import RecursiveCharacterTextSplitter
def chunk_documents(documents, chunk_size=1000, chunk_overlap=200):
    """
    Splits documents into smaller chunks.

    Args:
        documents (list): List of documents to be chunked.
        chunk_size (int): Size of each chunk.
        chunk_overlap (int): Overlap between chunks.
        
    Returns:
        list: List of text chunks.
    """
    text_splitter = RecursiveCharacterTextSplitter(
        chunk_size=chunk_size,
        chunk_overlap=chunk_overlap,
        length_function=len
    )
    return text_splitter.split_documents(documents)

# Chunk the documents
chunked_documents = chunk_documents(documents)
chunked_documents

NameError: name 'documents' is not defined

In [6]:
# indexing the documents using QDrant
from langchain_qdrant import QdrantVectorStore
from langchain_nvidia_ai_endpoints import NVIDIAEmbeddings
import os, dotenv
from datetime import datetime
dotenv.load_dotenv()

QDRANT_URL = os.getenv("QDRANT_URL", "http://localhost:6333")
EMBEDDING_MODEL_NAME = "nv-embed-v1"

def index_documents(documents, collection_name="pptx_collection"+str(int(datetime.now().timestamp())), qdrant_url=QDRANT_URL, embedding_model_name=EMBEDDING_MODEL_NAME):
    """
    Indexes documents using QDrant vector store.

    Args:
        documents (list): List of documents to be indexed.
        collection_name (str): Name of the QDrant collection.
        qdrant_url (str): URL of the QDrant instance.
        embedding_model_name (str): Name of the embedding model to use.

    Returns:
        Qdrant: The indexed QDrant vector store.
    """
    vectorstore = QdrantVectorStore.from_documents(
        documents,
        embedding=NVIDIAEmbeddings(model_name=embedding_model_name, nvidia_api_key=os.getenv("NVIDIA_API_KEY")),
        collection_name=collection_name,
        url=qdrant_url,
        prefer_grpc=True,
    )
    return vectorstore

# get vectorstore
def get_vectorstore(collection_name="pptx_collection", qdrant_url=QDRANT_URL, embedding_model_name=EMBEDDING_MODEL_NAME):
    """
    get vectorstore with specified collection name and QDrant URL.

    Returns:
        Qdrant: The QDrant vector store.
    """
    return QdrantVectorStore.from_existing_collection(
        collection_name=collection_name,
        url=qdrant_url,
        embedding=NVIDIAEmbeddings(model_name=embedding_model_name),
    )

In [43]:
# Example usage:
vectorstore = index_documents(chunked_documents, collection_name="genai")
print(f"Indexed {len(chunked_documents)} documents into QDrant collection '{vectorstore.collection_name}'.")

/home/benhima/pi/ai-services/.venv/lib/python3.12/site-packages/langchain_qdrant/qdrant.py:808: UserWarning: Qdrant client version 1.14.2 is incompatible with server version 1.10.1. Major versions should match and minor version difference must not exceed 1. Set check_compatibility=False to skip version check.
  client = QdrantClient(**client_options)


Indexed 10 documents into QDrant collection 'genai'.


In [7]:
# get the vectorstore
vectorstore = get_vectorstore(collection_name="genai")
print(f"Retrieved vectorstore with collection name: {vectorstore.collection_name}")

/home/benhima/pi/ai-services/.venv/lib/python3.12/site-packages/langchain_qdrant/qdrant.py:397: UserWarning: Qdrant client version 1.14.2 is incompatible with server version 1.10.1. Major versions should match and minor version difference must not exceed 1. Set check_compatibility=False to skip version check.
  client = QdrantClient(


Retrieved vectorstore with collection name: genai


In [45]:
# =============================================================================
# Multi-Agent Lab Generation Workflow
#
# Roles:
#  - planner_agent: decide QCM vs coding exercise and difficulty level
#  - retriever_agent: split documents, embed chunks, retrieve relevant ones
#  - qcm_generator_agent: generate MCQ questions/options/correct_answer
#  - code_generator_agent: produce coding exercise description and stub
#  - test_generator_agent: create unit tests: input/output pairs
#  - executor_agent: run the code against tests and capture results
#  - evaluator_agent: check test outcomes; if failure, trigger refinement loop
#
# Workflow:
#  1. load_documents(paths: List[str]) -> raw_texts
#  2. chunk_texts(raw_texts) -> chunks
#  3. embed_and_store(chunks) in vector DB
#  4. planner = planner_agent(user_query, metadata)
#     if planner.task == "qcm":
#         chunks = retriever_agent(planner.topic)
#         qcms = qcm_generator_agent(chunks, planner.difficulty)
#         evaluator_agent.validate_qcm(qcms)
#     elif planner.task == "code":
#         chunks = retriever_agent(planner.topic)
#         stub = code_generator_agent(chunks, planner.difficulty)
#         tests = test_generator_agent(stub, chunks)
#         result = executor_agent.run_tests(stub, tests)
#         evaluator_agent.loop_until_pass(stub, tests, result)
#
# Each agent should be implemented as a separate function or class method.
# Use LangChain + LangGraph for orchestration; vector DB for chunk retrieval;
# code sandbox for execution.
#
# Convention:
#  - Clear function signatures for each agent
#  - Use meaningful names and type hints
#  - Keep each agent focused on its responsibility
#
# Goals:
#  - Modular, testable, and easy-to-debug pipeline
#  - Support iterative refinement via evaluator loops
# =============================================================================

In [42]:
import os, json
from typing import Dict
from langchain_qdrant import Qdrant
from langchain_nvidia_ai_endpoints import ChatNVIDIA
from langchain_core.tools import tool
from langchain_core.messages import SystemMessage, HumanMessage
from langgraph.prebuilt import create_react_agent

MODEL_NAME = "meta/llama-3.3-70b-instruct"

# --- QCM generator using ReAct agent ---
def qcm_pipeline(user_query: str, vectorstore: Qdrant, difficulty: str, context: str, number_of_questions: int) -> Dict:
    llm = ChatNVIDIA(model=MODEL_NAME, nvidia_api_key=os.getenv("NVIDIA_API_KEY"))


    # Pre-fetch context (5 docs max)
    docs = vectorstore.similarity_search(user_query, k=5)
    context_text = "\n".join([d.page_content for d in docs])

    # Prepare prompt with explicit context
    sys_msg = SystemMessage(content=(
        f"You are an educational assistant. Using the context below, generate exactly {number_of_questions} multiple-choice questions at the '{difficulty}' level. "
        f"Each question must have exactly 3 plausible options and ONLY ONE correct answer.\n\n"
        f"Context:\n{context_text}\n{context}\n\n"
        "Return only a valid JSON in this exact format (use double quotes for all keys and string values):\n"
        '''{
    "quiz": [
        {
        "question": "string",
        "options": ["option1", "option2", "option3"],
        "answer": 1
        }
        // Repeat this format for each question
    ]
    }'''
        f"\nYou MUST return exactly {number_of_questions} questions inside the 'quiz' array. No more, no less. Do NOT explain. Only output the JSON."
    ))


    human_msg = HumanMessage(content=f"Generate a question about: {user_query}")

    # No retrieval tool needed here, just pass the LLM
    agent = create_react_agent(model=llm, tools=[], debug=True)

    result = agent.invoke({"messages": [sys_msg, human_msg]})

    try:
        parsed = json.loads(result['messages'][-1].content.strip().strip("```json").strip("```"))
    except json.JSONDecodeError:
        raise ValueError(f"Output is not valid JSON:\n{result['messages'][-1].content}")

    return parsed



In [43]:
qcm = qcm_pipeline("generative AI", vectorstore, difficulty="medium", number_of_questions=10, context="")

[-1:checkpoint] State at the end of step -1:
{'messages': []}
[0:tasks] Starting 1 task for step 0:
- __start__ -> {'messages': [SystemMessage(content='You are an educational assistant. Using the context below, generate exactly 10 multiple-choice questions at the \'medium\' level. Each question must have exactly 3 plausible options and ONLY ONE correct answer.\n\nContext:\nIntroduction to Generative AI\n\n\n\nKhaoula ALLAK\n\nGDG Mentor\n\n\n\nTable of Contents\n\n01\n\nWhat is Generative AI ?\n\n02\n\nFundamentals of Large Language Models \n\n03\n\nHow to customize the LLM  ? \n\n04\n\nPractice\n\n\n\n01\n\nWhat is Generative AI ?\n\n\n\nEvolution of AI \n\nWhat matters\n\n to us today !\n\n\n\nEvolution of AI Use Cases\n\nPredictive AI\n\nGenerative AI\n\nMultimodal\n\nGenerative AI\n\nText, Image & Code Generation\n\nText & Code Rewriting & Formatting\n\nSummarization\n\nExtractive Q&A\n\nImage & Video Descriptions\n\nRegression & Classification\n\nForecasting\n\nSentiment Analysis\

In [44]:
len(qcm['quiz'])

10

In [19]:
qcm['quiz']

[{'question': 'What type of AI is capable of generating text, images, and code?',
  'options': ['Predictive AI', 'Generative AI', 'Multimodal AI'],
  'answer': 1},
 {'question': 'Which of the following is a characteristic of Generative AI?',
  'options': ['Limited to predictive analytics',
   'Only generates text',
   'Can generate text, image, and code'],
  'answer': 2},
 {'question': 'What is the primary function of a Large Language Model (LLM)?',
  'options': ['To recognize and interpret human language',
   'To predict and generate the next word',
   'To analyze and understand visual data'],
  'answer': 0},
 {'question': 'What is an example of a Generative AI application?',
  'options': ['Sentiment analysis',
   'Image classification',
   'Text generation and rewriting'],
  'answer': 2},
 {'question': 'Which company published the Transformer paper in 2017?',
  'options': ['Google', 'Facebook', 'Microsoft'],
  'answer': 0},
 {'question': 'What is the name of the chatbot announced by 

In [113]:
# create a react agent that generate a coding exo question, list of input and expected outputs (to use later for verification)
# to check, is the user code correct or not?
# generate code (solution) for the exo.
# use a tool to run the code with the inputs and verify the outputs to `check if the agent's code or inputs outputs are correct`
# stop when the code is correct, and inputs and outputs are valid,
# return json format {exo: "", solution: "", inputs: [], outputs: []}
# using langchain Sandbox a code execution environment and react agent to generate coding exercises

from langchain_sandbox import PyodideSandboxTool

@tool
def verify_code(data: dict) -> dict:
    """
    Validate user code by running test cases.
    Args:
        data: A dict with keys 'inputs', 'outputs', 'solution'
    Returns:
        A dict with keys 'correct' (bool) and 'comment' (str)
    """
    return check_code(data["inputs"], data["outputs"], data["solution"])


def coding_exo_pipeline(user_query, vectorstore, difficulty, context):
    docs = vectorstore.similarity_search(user_query, k=5)
    context_text = "\n".join([d.page_content for d in docs])
    context_text=""
    llm = ChatNVIDIA(model=MODEL_NAME, nvidia_api_key=os.getenv("NVIDIA_API_KEY"))

    sys_msg = SystemMessage(content=(
        f"You are an expert Python coding exercise generator and validator.\n\n"
        f"Context:\n{context}\n{context_text}\n\n"
        f"Your task is to:\n"
        f"1. Generate a Python coding exercise at the '{difficulty}' level.\n"
        f"2. The exercise **must include** the function signature with all parameters and their types, and specify the return type.\n"
        f"3. Provide a correct Python solution for the exercise.\n"
        f"4. Create at least 3 test cases as a list of inputs (arguments for the function) and the expected output for each input.\n"
        f"5. Use the tool `verify_code` to check the correctness of your solution by passing your solution, inputs, and expected outputs.\n"
        f"6. If `verify_code` returns incorrect, regenerate the solution and/or test cases and recheck.\n"
        f"7. Repeat until the solution passes all test cases and `verify_code` returns correct.\n"
        f"8. Ensure a strict 1:1 mapping between inputs and outputs (same length, same order).\n"
        f"9. Return only a valid **JSON object** in this format:\n"
        '''{{
        "exercise": "string (clearly state the problem and the exact function signature, including parameter and return types)",
        "solution": "string (Python code)",
        "inputs": [[...], [...], ...],  # list of input arguments
        "outputs": [...]  # expected results (int, str, list, etc.)
        }}'''
        f"\nDO NOT include explanations, markdown, function calls, or code blocks. Only return the raw JSON."
    ))


    human_msg = HumanMessage(content=f"Generate a coding exercise about: {user_query}")
    tools = [verify_code]
    agent = create_react_agent(model=llm, tools=tools, debug=True)
    for _ in range(5):
        result = agent.invoke({"messages": [sys_msg, human_msg]})
        try:
            content = result['messages'][-1].content.strip().strip("```json").strip("```")
            parsed = json.loads(content)
            return parsed
        except Exception:
            raise Exception("failed to generate an appropriate json")

In [ ]:
# instlall deno in your terminal: curl -fsSL https://deno.land/install.sh | sh
# export DENO_INSTALL="$HOME/.deno"
# export PATH="$DENO_INSTALL/bin:$PATH"
# source ~/.bashrc 
# deno --version
# then run this command if it fails. quit you ide or reopen it and execute this cell.
# import subprocess
# print(subprocess.run(["deno", "--version"], capture_output=True).stdout.decode())

In [128]:
result = coding_exo_pipeline("function to divide two numbers", vectorstore, "easy", "")
result

[-1:checkpoint] State at the end of step -1:
{'messages': []}
[0:tasks] Starting 1 task for step 0:
- __start__ -> {'messages': [SystemMessage(content='You are an expert Python coding exercise generator and validator.\n\nContext:\n\n\n\nYour task is to:\n1. Generate a Python coding exercise at the \'easy\' level.\n2. The exercise **must include** the function signature with all parameters and their types, and specify the return type.\n3. Provide a correct Python solution for the exercise.\n4. Create at least 3 test cases as a list of inputs (arguments for the function) and the expected output for each input.\n5. Use the tool `verify_code` to check the correctness of your solution by passing your solution, inputs, and expected outputs.\n6. If `verify_code` returns incorrect, regenerate the solution and/or test cases and recheck.\n7. Repeat until the solution passes all test cases and `verify_code` returns correct.\n8. Ensure a strict 1:1 mapping between inputs and outputs (same length, 

{'exercise': 'Create a function called divide_numbers that takes two parameters, a and b, both of type float, and returns their division result as a float. The function should handle division by zero and return an error message in this case.',
 'solution': 'def divide_numbers(a: float, b: float) -> float:\n    if b == 0:\n        return "Error: Division by zero"\n    return a / b',
 'inputs': [[10.0, 2.0], [0.0, 5.0], [7.0, 0.0]],
 'outputs': [5.0, 0.0, 'Error: Division by zero']}

In [129]:
print(f"Exercise: {result['exercise']}")

Exercise: Create a function called divide_numbers that takes two parameters, a and b, both of type float, and returns their division result as a float. The function should handle division by zero and return an error message in this case.


In [130]:
print(result['solution'])

def divide_numbers(a: float, b: float) -> float:
    if b == 0:
        return "Error: Division by zero"
    return a / b


In [131]:
print(f"Exercise: {result['inputs']}")

Exercise: [[10.0, 2.0], [0.0, 5.0], [7.0, 0.0]]


In [132]:
print(f"Exercise: {result['outputs']}")

Exercise: [5.0, 0.0, 'Error: Division by zero']


In [133]:
from typing import List
def generate_lab(context: str, number_of_question: int, user_query: str=None, task: str="qcm", difficulty: str="easy", files_paths: List[str]=[]):
    if(task not in ["qcm", "code"]):
        raise ValueError("Invalid task type. Choose 'qcm' or 'code'.")
    # Load and parse documents
    documents = []
    for file_path in files_paths:
        try:
            docs = parse_file(file_path)
            documents.extend(docs)
        except Exception as e:
            print(f"Error parsing {file_path}: {e}")
    if not documents:
        raise ValueError("No valid documents found to process.")
    # Chunk the documents
    chunked_documents = chunk_documents(documents)
    if not chunked_documents:
        raise ValueError("No valid chunks created from documents.")
    collection_name = files_paths[0].split('/')[-1].split('.')[0] + "_collection"
    # Index the documents
    vectorstore = index_documents(chunked_documents, collection_name=collection_name)
    # use the appropriate agent based on the task
    if task == "qcm":
        # results is in a dict
        results = qcm_pipeline(user_query, vectorstore, difficulty, context, number_of_question)
    else:
        results = coding_exo_pipeline(user_query, vectorstore, difficulty, context)
    return results

In [134]:
import os
import json
from typing import Any, List
from langgraph.prebuilt import create_react_agent
from langchain.schema import SystemMessage, HumanMessage
from langchain_sandbox import PyodideSandboxTool
from langchain_nvidia import ChatNVIDIA
import re
import asyncio


def check_code(inputs: list, outputs: list, user_code: str) -> dict:
    """
    Runs user's function against test inputs and outputs using PyodideSandboxTool.
    Returns dict with correctness and comment.
    """

    tool = PyodideSandboxTool(allow_net=True)

    # Extract function name from user_code
    match = re.search(r'def (\w+)\s*\(', user_code)
    if not match:
        return {
            "correct": False,
            "comment": "### ❌ Problem Identified\nNo function definition found in user code."
        }
    func_name = match.group(1)

    for i, input_args in enumerate(inputs):
        try:
            # Combine function definition and function call in one execution
            call_str = (
                f"{user_code}\n"
                f"result = {func_name}(*{input_args})\n"
                f"result"
            )
            user_output = tool.invoke(call_str)

            if user_output != outputs[i]:
                return {
                    "correct": False,
                    "comment": f"### ❌ Problem Identified\nTest case {i} failed: expected {outputs[i]}, got {user_output}"
                }
        except Exception as e:
            return {
                "correct": False,
                "comment": f"### ❌ Problem Identified\nError running test case {i}.\n\n### 🧠 Error:\n{repr(e)}"
            }

    return {
        "correct": True,
        "comment": "### ✅ Solution Analysis\nAll test cases passed."
    }
def correct_pipeline(exercise: str, user_solution: str, inputs: Any, outputs: Any) -> dict:
    llm = ChatNVIDIA(model=MODEL_NAME, nvidia_api_key=os.getenv("NVIDIA_API_KEY"))

    sys_msg = SystemMessage(content=(
        "You are a Python expert. Your task is to verify if the user's solution is correct for a given coding exercise.\n\n"
        "Inputs:\n"
        "- A coding exercise description\n"
        "- A user-provided Python function solution\n"
        "- A list of test inputs (list of argument lists)\n"
        "- A list of expected outputs\n\n"
        "Instructions:\n"
        "1. Use the provided test framework to execute the user's solution.\n"
        "If the solution is correct:\n"
        "- Return a JSON object:\n"
        "{\n   \"correct\": true,\n   \"comment\": \"### ✅ Solution Analysis\\nExplain why the solution works correctly.\\n\\n### 💡 Suggestions for Improvement\\nMention any ways to make the code better: readability, performance, or Pythonic style.\"\n}\n\n"
        "If the solution is incorrect:\n"
        "- Return a JSON object:\n"
        "{\n  \"correct\": false,\n   \"comment\": \"### ❌ Problem Identified\\nExplain the bug or mistake in the user's solution.\\n\\n### 🧠 Why It Failed\\nDescribe clearly why it produces the wrong result.\\n\\n### 🛠️ How to Fix or Improve\\nGive guidance on how to fix the code and why the fix works.\"\n}\n\n"
        "⚠️ Only return a valid JSON with exactly two keys: 'correct' and 'comment'.\n"
        "⚠️ Do NOT include any text outside the JSON."
    ))

    human_msg = HumanMessage(content=(
        f"Exercise:\n{exercise}\n\n"
        f"User solution:\n{user_solution}\n\n"
        f"Inputs: {inputs}\n"
        f"Expected Outputs: {outputs}"
    ))

    # Run test locally first using check_code
    test_result = check_code(inputs, outputs, user_solution)

    return test_result


In [135]:
def check_code(inputs: list, outputs: list, user_code: str) -> dict:
    from langchain_sandbox import PyodideSandboxTool
    import re

    tool = PyodideSandboxTool(allow_net=True)

    match = re.search(r'def (\w+)\s*\(', user_code)
    if not match:
        return {
            "correct": False,
            "comment": "### ❌ Problem Identified\nNo function definition found in user code."
        }
    func_name = match.group(1)

    for i, input_args in enumerate(inputs):
        try:
            args = input_args if isinstance(input_args, (list, tuple)) else [input_args]
            call_str = f"{user_code}\nprint({func_name}(*{repr(args)}))"

            user_output = tool.invoke(call_str)
            try:
                user_output = eval(user_output.strip())
            except:
                pass

            if user_output != outputs[i]:
                return {
                    "correct": False,
                    "comment": f"### ❌ Problem Identified\nTest case {i} failed: expected {outputs[i]}, got {user_output}"
                }
        except Exception as e:
            return {
                "correct": False,
                "comment": f"### ❌ Problem Identified\nError running test case {i}.\n\n### 🧠 Error:\n{repr(e)}"
            }

    return {
        "correct": True,
        "comment": "### ✅ Solution Analysis\nAll test cases passed."
    }


In [136]:
res = check_code(result['inputs'], result['outputs'], result['solution'])
res

{'correct': True, 'comment': '### ✅ Solution Analysis\nAll test cases passed.'}

In [137]:
from langchain_nvidia import ChatNVIDIA
from langchain_core.messages import SystemMessage, HumanMessage
from langgraph.prebuilt import create_react_agent

MODEL_NAME = "meta/llama-3.3-70b-instruct"

def evaluate_and_feedback(exercise, solution, inputs, outputs):
    # 1. Run check_code
    check_result = check_code(inputs, outputs, solution)
    correct = check_result["correct"]
    comment = check_result["comment"]

    # 2. Prepare feedback prompt for the agent
    llm = ChatNVIDIA(model=MODEL_NAME, nvidia_api_key=os.getenv("NVIDIA_API_KEY"))
    sys_msg = SystemMessage(content=(
        "You are a Python tutor. Given the following exercise, user solution, test results, and feedback, provide constructive feedback to the user.\n\n"
        "Respond in Markdown. If the code is correct, praise the user and suggest improvements. If incorrect, explain the mistake and how to fix it."
    ))
    human_msg = HumanMessage(content=(
        f"Exercise:\n{exercise}\n\n"
        f"User solution:\n{solution}\n\n"
        f"Inputs: {inputs}\n"
        f"Expected Outputs: {outputs}\n"
        f"Test Results: {comment}\n"
    ))

    # 3. Use LangGraph ReAct agent for feedback
    agent = create_react_agent(model=llm, tools=[], debug=False)
    result = agent.invoke({"messages": [sys_msg, human_msg]})
    feedback = result['messages'][-1].content

    # 4. Return JSON
    return {
        "correct": correct,
        "feedback": feedback
    }

In [138]:
response = evaluate_and_feedback(
    exercise=result['exercise'],
    solution=result['solution'],
    inputs=result['inputs'],
    outputs=result['outputs']
)
print(response)

{'correct': True, 'feedback': '### Solution Analysis\nThe provided solution is correct and meets all the requirements specified in the exercise. Here\'s why:\n\n* The function `divide_numbers` takes two parameters `a` and `b` of type `float`.\n* It checks if `b` is zero before performing the division.\n* If `b` is zero, it returns an error message \'Error: Division by zero\'.\n* If `b` is not zero, it returns the division result as a float.\n\nHowever, there are a few potential improvements that could be suggested:\n\n* Instead of returning a string error message, consider raising a `ZeroDivisionError` exception. This is a more Pythonic way of handling errors and allows the caller to decide how to handle the exception.\n* Consider adding a docstring to the function to explain what it does, what parameters it takes, and what it returns.\n* The function could be made more robust by checking if the inputs are actually floats, and raising an error if they are not.\n\nHere\'s an example of 

In [139]:
# display markdown text inside the res['comment']
from IPython.display import display, Markdown

display(Markdown(response['feedback']))

### Solution Analysis
The provided solution is correct and meets all the requirements specified in the exercise. Here's why:

* The function `divide_numbers` takes two parameters `a` and `b` of type `float`.
* It checks if `b` is zero before performing the division.
* If `b` is zero, it returns an error message 'Error: Division by zero'.
* If `b` is not zero, it returns the division result as a float.

However, there are a few potential improvements that could be suggested:

* Instead of returning a string error message, consider raising a `ZeroDivisionError` exception. This is a more Pythonic way of handling errors and allows the caller to decide how to handle the exception.
* Consider adding a docstring to the function to explain what it does, what parameters it takes, and what it returns.
* The function could be made more robust by checking if the inputs are actually floats, and raising an error if they are not.

Here's an example of how the function could be improved:

```python
def divide_numbers(a: float, b: float) -> float:
    """
    Divide two numbers.

    Args:
        a (float): The dividend.
        b (float): The divisor.

    Returns:
        float: The division result.

    Raises:
        ZeroDivisionError: If the divisor is zero.
        TypeError: If the inputs are not floats.
    """
    if not isinstance(a, (int, float)) or not isinstance(b, (int, float)):
        raise TypeError("Both inputs must be numbers")
    if b == 0:
        raise ZeroDivisionError("Cannot divide by zero")
    return a / b
```

Overall, the original solution is correct, but there are opportunities to make it more robust and Pythonic.

In [ ]:
# {
#     'context': Optional[str],
#     'user_query': str,
#     'type': str,  # 'qcm' or 'code'
#     'difficulty': str,  # 'easy', 'medium', 'hard'
#     'number_of_questions': int,  # number of questions to generate
#     'files_paths': Optional[List[str]],  # paths to files to parse
# }

# # ila kan task == "qcm":
# {
# "quiz": [
#     {
#     "question": "string",
#     "options": ["option1", "option2", "option3"],
#     "answer": 1
#     }
# ]
# }

# # ila kan task == "code":
# {
#     'question': str,  # description of the coding exercise
#     'solution': str,  # code solution
#     'inputs': List[str],  # list of inputs for the code
#     'outputs': List[str]  # expected outputs for the inputs
# }

{}